# MERGING

In [2]:
import pandas as pd
import os

## data 자료 형태 파악하기
dir = "./data/" # 현재 디렉토리의 data 파일 안에 csv파일들 들어있으므로 편의상 변수로 지정

DIVIDER = '\n----------------------------------------------------------------\n' # 프린트했을 때 보기 편하게 선 긋기
listdir = [f for f in os.listdir(dir) if not f.startswith('.')]
print(listdir) # 현재 디렉토리의 data 파일 안에 있는 파일명 확인

['olist_sellers_dataset.csv', 'product_category_name_translation.csv', 'olist_orders_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_products_dataset.csv']


In [6]:
## 9개 csv파일 데이터 읽기
cust = pd.read_csv(dir + 'olist_customers_dataset.csv')
geoloc = pd.read_csv(dir + 'olist_geolocation_dataset.csv')
order_items = pd.read_csv(dir + 'olist_order_items_dataset.csv')
payments = pd.read_csv(dir + 'olist_order_payments_dataset.csv')
reviews = pd.read_csv(dir + 'olist_order_reviews_dataset.csv')
orders = pd.read_csv(dir + 'olist_orders_dataset.csv')
products = pd.read_csv(dir + 'olist_products_dataset.csv')
sellers = pd.read_csv(dir + 'olist_sellers_dataset.csv')
translation = pd.read_csv(dir + 'product_category_name_translation.csv')



## 물품카테고리명은 영어로 미리 변경

In [7]:
products = products.merge(translation, on='product_category_name').drop('product_category_name', axis=1)

## 각각 merging

In [8]:


total_df = pd.merge(orders, order_items, on='order_id')
total_df = pd.merge(total_df, payments, on='order_id')
total_df = pd.merge(total_df, reviews, on='order_id')
total_df = pd.merge(total_df, cust, on='customer_id')
total_df = pd.merge(total_df, products, on='product_id')
total_df = pd.merge(total_df, sellers, on='seller_id')
#total_df = pd.merge(total_df, geoloc, how='left',left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix')


## geolocation을 customer, seller 두가지와 각각 JOIN

### state city 이름 cleaning

In [9]:
## geolocation dataset과 join
total_df['customer_state'] = total_df['customer_state'].apply(lambda x : x.lower())
total_df['customer_city'] = total_df['customer_city'].apply(lambda x : x.lower())
total_df['seller_state'] = total_df['seller_state'].apply(lambda x : x.lower())
total_df['seller_city'] = total_df['seller_city'].apply(lambda x : x.lower())
geo_state = geoloc.groupby('geolocation_state')['geolocation_lat','geolocation_lng'].mean().reset_index()
geo_state['state'] = geo_state['geolocation_state'].apply(lambda x : x.lower())
geo_city = geoloc.groupby('geolocation_city')['geolocation_lat','geolocation_lng'].mean().reset_index()
geo_city['city'] = geo_city['geolocation_city'].apply(lambda x : x.lower())

geo_city.rename(columns={'geolocation_lat': 'c_lat','geolocation_lng':'c_lng'}, inplace=True)

missing_geo = geo_state.isnull().sum()
for i in range(len(missing_geo)):
    print("Missing rows in {} : {}".format(missing_geo.index[i], missing_geo.values[i]))


Missing rows in geolocation_state : 0
Missing rows in geolocation_lat : 0
Missing rows in geolocation_lng : 0
Missing rows in state : 0


### customer 와 join

In [ ]:

total_df = pd.merge(total_df, geo_state, how='left', left_on='customer_state',right_on='state')
total_df = pd.merge(total_df, geo_city, how='left',left_on='customer_city',right_on='city')


### seller와 join

In [10]:

total_df = pd.merge(total_df, geo_state, how='left', left_on='seller_state',right_on='state')
total_df = pd.merge(total_df, geo_city, how='left',left_on='seller_city',right_on='city')


### csv로 저장

In [11]:
print(total_df.head())
total_df.info()
total_df.to_csv('total_df.csv')

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
2  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
3  128e10d95713541c87cd1a2e48201934  a20e8105f23924cd00833fd87daa0831   
4  0e7e841ddf8f8f2de2bad69267ecfbcf  26c7ac168e1433912a51b924fbd34d34   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
2    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
3    delivered      2017-08-15 18:29:31  2017-08-15 20:05:16   
4    delivered      2017-08-02 18:24:47  2017-08-02 18:43:15   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2017-10-04 19:55:00           2017-10